In [5]:
file_path = './emails.csv'
lines = []
with open(file_path, 'r') as file:
    for line in file:
        lines.append(line.strip())
print(lines[1])

"Hello Innovator, What if we told you that you could work with your dream company straight out of college? Yes! You heard it right! Participate in the Accenture Innovation Challenge to win big and you may also get an opportunity to fast-track your recruitment process journey with Accenture. Application Link: https://unstop.com/o/l58dzh1/?ref=h1FyMTt0 Roles Offered: Associate Software Engineer System and Application Services Associate Rewards: ❖ Prizes worth INR 1,00,000 per team member❖ Fast-Track Interview Opportunity Eligibility: Undergraduate and Postgraduate students Participate in the most sought-after challenge, and let your ideas make waves. Note that the fast-track recruitment process will be offered to all the eligible (as per our recruitment process criteria) participants who would submit their innovative ideas. Eligible participants will receive communications about the recruitment process from Accenture. Best Regards,Team Unstop Unsubscribe here"
